In [1]:
print("ok")

ok


In [2]:
%pwd

'd:\\Deployed\\GenAI\\research'

In [3]:
import os 
os.chdir("../")

In [4]:
%pwd

'd:\\Deployed\\GenAI'

In [44]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [45]:
# Extract text from PDF files
def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [46]:
extracted_data = load_pdf_files("data")

In [47]:
extracted_data

[Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2025-08-02T09:51:56+05:30', 'author': 'shilpa mehta', 'moddate': '2025-08-02T09:51:56+05:30', 'source': 'data\\College-Rule.pdf', 'total_pages': 36, 'page': 0, 'page_label': '1'}, page_content='Regulation No.: PU/AC-26/6/07_2025                                                                              1 \n \n  \n \nAcademic Regulations'),
 Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2025-08-02T09:51:56+05:30', 'author': 'shilpa mehta', 'moddate': '2025-08-02T09:51:56+05:30', 'source': 'data\\College-Rule.pdf', 'total_pages': 36, 'page': 1, 'page_label': '2'}, page_content='Regulation No.: PU/AC-26/6/07_2025                                                                              2 \n \n \n \n \n \n \n \n \n \n \n \n \n \nRegulation No.: PU/AC-26/6/07_2025 \n \nResolution No. 26.6 of the 26th Meeting of the Acad

In [48]:
len(extracted_data)

559

In [49]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [50]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [51]:
minimal_docs

[Document(metadata={'source': 'data\\College-Rule.pdf'}, page_content='Regulation No.: PU/AC-26/6/07_2025                                                                              1 \n \n  \n \nAcademic Regulations'),
 Document(metadata={'source': 'data\\College-Rule.pdf'}, page_content='Regulation No.: PU/AC-26/6/07_2025                                                                              2 \n \n \n \n \n \n \n \n \n \n \n \n \n \nRegulation No.: PU/AC-26/6/07_2025 \n \nResolution No. 26.6 of the 26th Meeting of the Academic Council held on \n25/07/2025 and ratified by the Board of Management in its Meeting held on \n28/07/2025 \n \n \n              \n \n \nJuly 2025 \n \n \n \n \n \n \n \n \nAcademic Regulations'),
 Document(metadata={'source': 'data\\College-Rule.pdf'}, page_content='Regulation No.: PU/AC-26/6/07_2025                                                                              3 \n \n \n \n \n \n \n \n \n \n \n \n[Left Blank intentionally]'),
 Document(me

In [52]:
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [53]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 2353


In [54]:
texts_chunk

[Document(metadata={'source': 'data\\College-Rule.pdf'}, page_content='Regulation No.: PU/AC-26/6/07_2025                                                                              1 \n \n  \n \nAcademic Regulations'),
 Document(metadata={'source': 'data\\College-Rule.pdf'}, page_content='Regulation No.: PU/AC-26/6/07_2025                                                                              2 \n \n \n \n \n \n \n \n \n \n \n \n \n \nRegulation No.: PU/AC-26/6/07_2025 \n \nResolution No. 26.6 of the 26th Meeting of the Academic Council held on \n25/07/2025 and ratified by the Board of Management in its Meeting held on \n28/07/2025 \n \n \n              \n \n \nJuly 2025 \n \n \n \n \n \n \n \n \nAcademic Regulations'),
 Document(metadata={'source': 'data\\College-Rule.pdf'}, page_content='Regulation No.: PU/AC-26/6/07_2025                                                                              3 \n \n \n \n \n \n \n \n \n \n \n \n[Left Blank intentionally]'),
 Document(me

In [55]:
from langchain.embeddings import HuggingFaceEmbeddings

def download_embeddings():
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name=model_name
    )
    return embeddings

embedding = download_embeddings()

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: e8a36d3e-72b7-49f0-b948-25bb3d602a7b)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json
Retrying in 1s [Retry 1/5].


In [56]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [57]:
vector = embedding.embed_query("Hello world")
vector

[-0.034477200359106064,
 0.031023219227790833,
 0.0067349993623793125,
 0.02610897459089756,
 -0.03936201333999634,
 -0.16030248999595642,
 0.06692397594451904,
 -0.006441470701247454,
 -0.04745051637291908,
 0.014758859761059284,
 0.07087533175945282,
 0.05552757531404495,
 0.019193289801478386,
 -0.026251329109072685,
 -0.010109508410096169,
 -0.026940548792481422,
 0.022307483479380608,
 -0.022226618602871895,
 -0.14969265460968018,
 -0.01749304309487343,
 0.007676269859075546,
 0.05435226485133171,
 0.003254482988268137,
 0.03172600269317627,
 -0.0846213772892952,
 -0.02940598875284195,
 0.051595672965049744,
 0.04812409356236458,
 -0.003314794274047017,
 -0.05827922374010086,
 0.041969284415245056,
 0.022210709750652313,
 0.1281888633966446,
 -0.022338956594467163,
 -0.011656241491436958,
 0.06292833387851715,
 -0.032876305282115936,
 -0.09122610837221146,
 -0.03117538057267666,
 0.05269957706332207,
 0.047034792602062225,
 -0.08420304954051971,
 -0.0300561785697937,
 -0.020744718

In [58]:
print( "Vector length:", len(vector))

Vector length: 384


In [59]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [60]:

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY


In [61]:
from pinecone import Pinecone 
print("Successfully")

Successfully


In [62]:
from pinecone import Pinecone 
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)

In [63]:
pc

In [ ]:
from pinecone import ServerlessSpec 

index_name = "presi-bot"

if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384,  # Dimension of the embeddings
        metric= "cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(index_name)

In [65]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=texts_chunk,
    embedding=embedding,
    index_name=index_name
)

In [ ]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding
)

In [67]:
# Add more data to the existing pinecone index
dswith = Document(
    page_content="Intellipaat is a youtube channel that provides tutorials on various topics.",
    metadata={"source": "Youtube"}
)

In [68]:
docsearch.add_documents(documents=[dswith])

['8a74a4ab-ccfd-49dd-864e-444b8f786b75']

In [69]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [70]:
retrieved_docs = retriever.invoke("What is Generative AI?")
retrieved_docs

[Document(id='2495761c-fd14-4f30-ad08-dcaf21171e05', metadata={'source': 'data\\Generative AI.pdf'}, page_content='generative AI. Far from being just another application field, AI \nagents are a burgeoning domain that amplifies the potential of gen-\nerative AI. They hold the promise of enhancing every application'),
 Document(id='b4eff945-47d8-42a5-86a5-4727d970849b', metadata={'source': 'data\\Generative AI.pdf'}, page_content='generative AI. Far from being just another application field, AI \nagents are a burgeoning domain that amplifies the potential of gen-\nerative AI. They hold the promise of enhancing every application'),
 Document(id='126b4ecc-ab66-4361-a6ac-ba6958ef5354', metadata={'source': 'data\\Generative AI.pdf'}, page_content='generative AI. Far from being just another application field, AI \nagents are a burgeoning domain that amplifies the potential of gen-\nerative AI. They hold the promise of enhancing every application')]

In [71]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-4o")

In [72]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [73]:
system_prompt = (
    "You are an Edcational assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [74]:
question_answer_chain = create_stuff_documents_chain(chatModel, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [75]:
response = rag_chain.invoke({"input": "what is Generative AI?"})
print(response["answer"])

Generative AI refers to technology that uses machine learning models to generate new content, such as text, images, or music, by learning patterns from existing data. It holds the potential to enhance various applications by creating new and innovative outputs. 


In [76]:
response = rag_chain.invoke({"input": "what is LLM?"})
print(response["answer"])

LLM stands for Large Language Model, which is a type of artificial intelligence designed to understand and generate human-like text by learning from large amounts of natural language data. They scale up without specific training for individual tasks, demonstrating adaptability by combining and extrapolating learned behaviors.


In [78]:
response = rag_chain.invoke({"input": "what is the E ATTENDANCE REQUIREMENTS?"})
print(response["answer"])

The E attendance requirements specify that a student must have a minimum attendance of 75% in all courses in each semester or academic term. Exceptions can be made for approved leave of absence, such as participating in competitions or experiencing medical emergencies, where relaxation may be granted. Additionally, these requirements must adhere to any government regulatory body prescriptions for specific programs.
